In [ ]:
import pandas as pd
from pathlib import Path
import os
import sys
import pickle

sys.path.append(str(Path().resolve().parent / "src"))

from ingest import load_cites_data
from mock import generate_mock_trafficking_data
from wtie.viz.map_routes import create_trade_routes_map

**Load and Filter Data**

In [ ]:
df_raw = pd.read_csv("../data/cites/sample.csv", low_memory=False)
df_legal = load_cites_data(df_raw)

df_legal.head()

**Load Synthetic Mock Data**

In [ ]:
df_synth = generate_mock_trafficking_data()
df_synth.head()

**Combine for Unified View**

In [ ]:
df_all = pd.concat([df_legal, df_synth], ignore_index=True)
df_all["synthetic"].value_counts()

**Top Importers/Exporters**

In [ ]:
df_all["Exporter"].value_counts().head(10)

In [ ]:
df_all["Importer"].value_counts().head(10)

**Most Common Trade Routes**

In [ ]:
df_all.groupby(["Exporter", "Importer", "synthetic"]).size().sort_values(
    ascending=False
).head(10)

**Most Common Species & Terms**

In [ ]:
df_all["Taxon"].value_counts().head(10)

In [ ]:
df_all["Term"].value_counts()

**Map**

In [ ]:
os.makedirs("outputs", exist_ok=True)

# Point to the project root and the flat file path
storage_path = Path("../storage").resolve()
file_path = storage_path / "combined_trade_data"

with open(file_path, "rb") as f:
    df = pickle.load(f)

output_path = Path("../outputs/trade_routes_map.html").resolve()

create_trade_routes_map(df, output_path=output_path)

In [ ]:
df.info()
df.head()